In [ ]:
from glob import glob
import os
import json
import csv
import subprocess

In [ ]:
%cd ..

In [ ]:
ckpts_root_dir = "checkpoints"
results_root_dir = "results"
test_results_summary_dir = "category-summaries"
visuals_folder_name = "images"

ckpt_category = "tumtraf-i"

test_results_filename = "test_results.json"
test_results_csv_filename = "test_results.csv"
benchmark_filename = "benchmark.json"
override_test_results = False
override_visuals = False
override_videos = False
override_benchmark = False
visuals_camera_bbox_score: float = 0.15
visuals_max_samples: int = None
visuals_save_bboxes = True
visuals_include_combined = True  # gt + pred

ckpt_dir = os.path.join(ckpts_root_dir, ckpt_category)
chkpts = glob(os.path.join(ckpt_dir, "*"))
results_w_category_dir = os.path.join(results_root_dir, ckpt_category)

test_results_compiled_filename = f"{ckpt_category}.csv"
test_results_compiled_path = os.path.join(
    results_root_dir,
    test_results_summary_dir,
    test_results_compiled_filename,
)

# Testing

---

In [ ]:
for x in chkpts:
    summary_path = os.path.join(
        results_w_category_dir,
        os.path.basename(x),
        test_results_filename,
    )

    if not override_test_results and os.path.exists(summary_path):
        print("Skipping, test results exist for", x)
        continue

    cfg_path = os.path.join(x, "configs.yaml")
    pth_path = os.path.join(x, "latest.pth")

    command = f"torchpack dist-run -np 1 python tools/test.py \
        {cfg_path} {pth_path} \
        --eval bbox --eval-options extensive_report=True \
        save_summary_path={summary_path} \
        > /dev/null 2>&1"
    
    print("Running tests for", x)
    os.system(command)


# Create images

---

In [ ]:
for x in chkpts:
    cfg_path = os.path.join(x, "configs.yaml")
    pth_path = os.path.join(x, "latest.pth")
    id = x.split("/")[-1]
    
    out_dir = os.path.join(results_w_category_dir, id, visuals_folder_name)
    bboxes_dir = os.path.join(results_w_category_dir, id, "bboxes-pred")
    labels_dir = os.path.join(results_w_category_dir, id, "labels-pred")

    if not override_visuals and os.path.exists(out_dir):
        size = subprocess.check_output(['du','-sh', out_dir]).split()[0].decode('utf-8')
        print(f"Skipping: visuals exist ({size}) for", x)
        continue

    command = f"torchpack dist-run -np 1 python tools/visualize.py \
        {cfg_path} --checkpoint {pth_path} \
        --out-dir {out_dir} \
        --mode pred --split test"
    
    if visuals_save_bboxes:
        command += " --save-bboxes --save-labels"
        command += f" --save-bboxes-dir {bboxes_dir}"
        command += f" --save-labels-dir {labels_dir}"
    
    if x.split("/")[-1][0] in ["C", "c"]:
        command += f" --bbox-score {visuals_camera_bbox_score}"

    if visuals_max_samples is not None:
        command += f" --max-samples {visuals_max_samples}"

    if visuals_include_combined:
        command += " --include-combined"

    command += " > /dev/null 2>&1"

    print("Creating images for", x)

    os.system(command)

    size = subprocess.check_output(['du','-sh', out_dir]).split()[0].decode('utf-8')
    print("\tDirectory size: " + size)

# Create videos

---

In [ ]:
for x in glob(os.path.join(results_w_category_dir, "*")):
    out_path = os.path.join(x, "videos")
    if not override_videos and os.path.exists(out_path):
        size = subprocess.check_output(['du','-sh', out_path]).split()[0].decode('utf-8')
        print(f"Skipping: videos exist ({size}) for", x)
        continue

    print("Creating videos for", x)
    
    for source_folder_dir in glob(os.path.join(x, visuals_folder_name, "*")):
        folder_name = os.path.basename(source_folder_dir)
        if not os.path.isdir(source_folder_dir):
            continue
        target_path = os.path.join(out_path, folder_name + ".mp4")
        command = f"python tools/visualization/create_video.py -s {source_folder_dir} -t {target_path}"
        os.system(command)

# Benchmark

---

In [ ]:
for x in chkpts:
    target_path = os.path.join(
            results_w_category_dir,
            os.path.basename(x),
            benchmark_filename,
        )

    cfg_path = os.path.join(x, "configs.yaml")
    pth_path = os.path.join(x, "latest.pth")

    if not override_benchmark and os.path.exists(target_path):
        print(f"Skipping: benchmark exists for", x)
        continue

    print("Benchmarking for", x)
    command = f"python tools/benchmark.py {cfg_path} {pth_path} --out {target_path}"
    command += " > /dev/null 2>&1"
    os.system(command)

# Compiling

---

## Result formatting

In [ ]:
ERR_NAME_MAPPING = {
    "trans_err": "mATE",
    "scale_err": "mASE",
    "orient_err": "mAOE",
}

CLASSES = (
    "CAR",
    "TRAILER",
    "TRUCK",
    "VAN",
    "PEDESTRIAN",
    "BUS",
    "MOTORCYCLE",
    "BICYCLE",
    "EMERGENCY_VEHICLE",
    # "OTHER",
)

HEADERS = [
    "id",
    "eval_type",
    "sensors",
    "test_fps",
    "test_mem",
    "mAP",
    "mAOE",
    "mATE",
    "mASE",
    
]

for m in ["mAP", "mAOE", "mATE", "mASE"]:
    for x in CLASSES:
        HEADERS.append(f"{x}_{m}")

HEADERS.append("num_gt_bboxes")

for x in CLASSES:
    HEADERS.append(f"num_{x}")

# Compile individually

In [ ]:
for i, x in enumerate(glob(os.path.join(results_w_category_dir, "*"))):
    if not os.path.isdir(x):
        continue
    print("Compiling results for", x)

    id = os.path.basename(x)
    path = os.path.join(x, test_results_filename)
    with open(path, "rb") as json_file:
        data = json.load(json_file)
    
    out_csv = os.path.join(x, test_results_csv_filename)
    with open(out_csv, "w") as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writeheader()

        sensors = ""
        if "lid" in id.lower():
            sensors += "L"
        elif "cam" in id.lower():
            sensors += "C"
        else:
            sensors += "LC"

        # read benchmark data
        benchmark_path = os.path.join(x, benchmark_filename)
        benchmark_data = None
        with open(benchmark_path, "rb") as json_file:
            benchmark_data = json.load(json_file)

        for eval_type, eval_data in data.items():
            row = {
                "id": id,
                "sensors": sensors,
                "test_fps": benchmark_data["fps"],
                "test_mem": benchmark_data["memory_allocated"],
                "eval_type": eval_type,
                "mAP": eval_data["mean_ap"],
            }

            for x in CLASSES:
                if x in eval_data["class_counts"]:
                    row.update({f"num_{x}": eval_data["class_counts"][x]})
                else:
                    row.update({f"num_{x}": ""})

            for tp_name, tp_val in eval_data["tp_errors"].items():
                if tp_name in ERR_NAME_MAPPING:
                    row.update({ERR_NAME_MAPPING[tp_name]: tp_val})

            class_aps = eval_data["mean_dist_aps"]
            class_tps = eval_data["label_tp_errors"]

            for class_name in class_aps.keys():
                row.update({f"{class_name}_mAP": class_aps[class_name]})
                for tp_name, tp_val in class_tps[class_name].items():
                    if tp_name in ERR_NAME_MAPPING:
                        row.update(
                            {
                                f"{class_name}_{ERR_NAME_MAPPING[tp_name]}": class_tps[class_name][
                                    tp_name
                                ]
                            }
                        )

            writer.writerows([row])


## Compile into a summary

In [ ]:
os.makedirs(os.path.dirname(test_results_compiled_path), exist_ok=True)
with open(test_results_compiled_path, "w") as f:
    writer = csv.DictWriter(f, fieldnames=HEADERS)
    writer.writeheader()

    for i, x in enumerate(glob(os.path.join(results_w_category_dir, "*"))):
        id = os.path.basename(x)
        print("Compiling results for", x)
        data = None
        path = os.path.join(x, test_results_filename)
        with open(path, "rb") as json_file:
            data = json.load(json_file)

        sensors = ""
        if "lid" in id.lower():
            sensors += "L"
        elif "cam" in id.lower():
            sensors += "C"
        else:
            sensors += "LC"

        # read benchmark data
        benchmark_path = os.path.join(x, benchmark_filename)
        benchmark_data = None
        with open(benchmark_path, "rb") as json_file:
            benchmark_data = json.load(json_file)

        for eval_type, eval_data in data.items():
            row = {
                "id": id,
                "sensors": sensors,
                "test_fps": round(benchmark_data["fps"], 2),
                "test_mem": round(benchmark_data["memory_allocated"], 2),
                "eval_type": eval_type,
                "mAP": eval_data["mean_ap"],
            }

            for x in CLASSES:
                if x in eval_data["class_counts"]:
                    row.update({f"num_{x}": eval_data["class_counts"][x]})
                else:
                    row.update({f"num_{x}": ""})

            for tp_name, tp_val in eval_data["tp_errors"].items():
                if tp_name in ERR_NAME_MAPPING:
                    row.update({ERR_NAME_MAPPING[tp_name]: tp_val})

            class_aps = eval_data["mean_dist_aps"]
            class_tps = eval_data["label_tp_errors"]

            for class_name in class_aps.keys():
                row.update({f"{class_name}_mAP": class_aps[class_name]})
                for tp_name, tp_val in class_tps[class_name].items():
                    if tp_name in ERR_NAME_MAPPING:
                        row.update(
                            {
                                f"{class_name}_{ERR_NAME_MAPPING[tp_name]}": class_tps[class_name][
                                    tp_name
                                ]
                            }
                        )

            row.update({"num_gt_bboxes": eval_data["total_gt_bboxes"]})

            writer.writerows([row])